In [ ]:
import numpy as np  # 線形代数
import pandas as pd  # データ処理、CSVファイルのI/O（例：pd.read_csv）
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv("../input/nlp-getting-started/train.csv")

# "text" カラムのテキストデータをTF-IDFベクトルに変換
vectorizer = TfidfVectorizer(max_features=1000)
X_text_tfidf = vectorizer.fit_transform(df["text"]).toarray()

# 特徴量とターゲットに分割
X = df[["keyword", "location"]].copy()
X = pd.get_dummies(X, columns=["keyword", "location"], dummy_na=True)
X = np.hstack((X.values, X_text_tfidf))
y = df["target"]

# データを訓練セットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 学習されていないモデルオブジェクト
clf = RandomForestClassifier(random_state=42)

# モデルの訓練
clf.fit(X_train, y_train)

# テストセットで予測
y_pred = clf.predict(X_test)

# 精度の計算
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.79


In [3]:
# test.csv に対して予測を行う
df_prod = pd.read_csv("../input/nlp-getting-started/test.csv")
X_prod_text_tfidf = vectorizer.transform(df_prod["text"]).toarray()

# 訓練データと同じダミー変数構造を作成するため、元の訓練データの列名を取得
X_prod = df_prod[["keyword", "location"]].copy()
X_prod_dummies = pd.get_dummies(X_prod, columns=["keyword", "location"], dummy_na=True)

# 訓練データで使用した特徴量の列名を取得 (TF-IDF部分を除く)
train_feature_cols = pd.get_dummies(
    df[["keyword", "location"]], columns=["keyword", "location"], dummy_na=True
).columns

# 訓練データに存在してテストデータに存在しない列を特定
missing_cols = set(train_feature_cols) - set(X_prod_dummies.columns)

# 欠落した列をすべて一度に追加 (パフォーマンス警告を回避)
if missing_cols:
    missing_df = pd.DataFrame(0, index=X_prod_dummies.index, columns=list(missing_cols))
    X_prod_dummies = pd.concat([X_prod_dummies, missing_df], axis=1)

# テストデータに存在して訓練データに存在しない列を削除
X_prod_dummies = X_prod_dummies[train_feature_cols]

# TF-IDF特徴量と結合
X_prod = np.hstack((X_prod_dummies.values, X_prod_text_tfidf))
y_pred_prod = clf.predict(X_prod)

# 提出用ファイルの作成
submission = pd.DataFrame({"id": df_prod["id"], "target": y_pred_prod})
submission.to_csv("../output/submission.csv", index=False)